In [5]:
from __future__ import print_function
import keras
import tempfile
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger  #, UpdatePruningStep
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
import tensorflow_model_optimization as tfmot
import numpy as np
import time
import os
import pickle
from keras.datasets import cifar100
import tensorflow as tf

In [4]:
!pip install tensorflow_model_optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 12.3 MB/s 


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n' .join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 11 05:57:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    15W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 6.5 MB/s eta 0:00:0000:01


In [6]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [6]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [7]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

In [8]:
def tflite_conv(model,path,quant=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quant == True:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    pruned_tflite_model = converter.convert()
    with open(path, 'wb') as f:
      f.write(pruned_tflite_model)
    print('Saved pruned TFLite model to:',path)

In [9]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [12]:
def resnet_training(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default',block_size=(1,1)):

    # Default parameters
    batch_size = 128 
    epochs = 300
    data_augmentation = False #True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    ####Changes start#####
    num_images = x_train.shape[0] #* (1 - validation_split)

    if end_step == 'default':
        end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
    if poly:
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                      frequency=frequency),
            'block_size': block_size
        }
    if const:
            pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                       frequency=frequency),
                'block_size': block_size
        }

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    model = resnet_v1(input_shape=input_shape, depth=depth,num_classes=num_classes)
    model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

    model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname='/home/kal2257/saved_models/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=';')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback, pruning_callbacks.UpdatePruningStep(), tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)]

    st = time.time()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    #%tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

    #save pruned model
    model_for_export = tfmot.sparsity.keras.strip_pruning(model)
    pruned_keras_file = save_dir+'/PRUNE_'+model_path
    keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)
    
    #convert to tflite+ save
    pruned_tflite_file=save_dir+'/lite_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_file)
    
    #save to tf lite + qaunt
    pruned_tflite_quant_file=save_dir+'/lite_quant_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_quant_file,quant=True)
    
    pruned=get_gzipped_model_size(pruned_keras_file)
    tflite_pruned=get_gzipped_model_size(pruned_tflite_file)
    tflite_quant_pruned=get_gzipped_model_size(pruned_tflite_quant_file)
    
    
    print("Size of gzipped pruned Keras model: %.2f bytes" % (pruned)) 
    print("Size of gzipped pruned TFlite model: %.2f bytes" % (tflite_pruned))
    print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (tflite_quant_pruned))
    
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],pruned,tflite_pruned,tflite_quant_pruned], handle)

    return(model,model_path)

In [10]:
# Load the CIFAR10 data.
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()
# Normalize data.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Convert class vectors to binary class matrices.
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)

169001437/169001437 [==============================] - 6s 0us/step


# Experimenting with constant sparsity 
In order to see the effects of pruning on a network with a very large number of parameters,Resnet44, verses in network with fewer parameters like,Resnet20, we first investigated the effects of constant sparsity versus polynomials varsity scheduling.

Resnet44

In [ ]:
sparsity_level=[0.6,0.8,0.9]

names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_poly_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = 0.5, final_sparsity=sparsity_level[x],gpu="T4",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet44_poly_sparsity_sixty
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_288  (None, 32, 32, 16)  882         ['input_9[0][0]']                
  (PruneLowMagnitude)                                                                             
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_288[
 alization_272 (PruneLowMagnitu        

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 _303 (PruneLowMagnitude)                                        lization_303[0][0]']             
                                                                                                  
 prune_low_magnitude_conv2d_322  (None, 8, 8, 64)    73794       ['prune_low_magnitude_activation_
  (PruneLowMagnitude)                                            303[0][0]']                      
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 8, 8, 64)    257         ['prune_low_magnitude_conv2d_322[
 alization_304 (PruneLowMagnitu                                  0][0]']                          
 de)                                                                                              
                                                                                                  
 prune_low_magnitude_add_147 (P  (None, 8, 8, 64)    1           ['prune_low_magnitude_activation_
 runeLowMa

391/391 [==============================] - ETA: 0s - loss: 4.6090 - accuracy: 0.0711WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 73s 93ms/step - loss: 4.6090 - accuracy: 0.0711 - val_loss: 4.3551 - val_accuracy: 0.0892 - lr: 0.0010
Learning rate:  0.001
Epoch 2/300
391/391 [==============================] - ETA: 0s - loss: 3.7491 - accuracy: 0.1766WARNING:tensorflow:Can save best model only with val_acc available, skipping.


...

391/391 [==============================] - 32s 83ms/step - loss: 2.3145 - accuracy: 0.5520 - val_loss: 5.0161 - val_accuracy: 0.3429 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 199/300
391/391 [==============================] - ETA: 0s - loss: 2.3625 - accuracy: 0.5436WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 32s 83ms/step - loss: 2.3625 - accuracy: 0.5436 - val_loss: 5.0357 - val_accuracy: 0.3361 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 200/300
391/391 [==============================] - ETA: 0s - loss: 2.3621 - accuracy: 0.5416WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 32s 83ms/step - loss: 2.3621 - accuracy: 0.5416 - val_loss: 5.0745 - val_accuracy: 0.3300 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 201/300
391/391 [==============================] - ETA: 0s - loss: 2.3927 - accuracy: 0.5354WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 32s 83ms/step - loss: 2.3927 - accuracy: 0.5354 - val_loss: 5.0142 - val_accuracy: 0.3369 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 202/300
391/391 [==============================] - ETA: 0s - loss: 2.3994 - accuracy: 0.5371WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 32s 82ms/step - loss: 2.3994 - accuracy: 0.5371 - val_loss: 5.0103 - val_accuracy: 0.3373 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 203/300
 56/391 [===>..........................] - ETA: 27s - loss: 2.3933 - accuracy: 0.5428

In [ ]:
sparsity_level=[0.6,0.8,0.9]
names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_const_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = False, final_sparsity=sparsity_level[x],gpu="T4",\
                    const=True, poly=False,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet44_const_sparsity_sixty
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)


2022-12-11 06:01:42.723377: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 06:01:42.735416: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 06:01:42.738043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 06:01:42.741672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d (Pr  (None, 32, 32, 16)  882         ['input_1[0][0]']                
 uneLowMagnitude)                                                                                 
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d[0][0
 alization (PruneLowMagnitude)                                   ]']                              
                                                                        

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 alization_28 (PruneLowMagnitud                                  ][0]']                           
 e)                                                                                               
                                                                                                  
 prune_low_magnitude_add_13 (Pr  (None, 16, 16, 32)  1           ['prune_low_magnitude_activation_
 uneLowMagnitude)                                                26[0][0]',                       
                                                                  'prune_low_magnitude_batch_norma
                                                                 lization_28[0][0]']              
                                                                                                  
 prune_low_magnitude_activation  (None, 16, 16, 32)  1           ['prune_low_magnitude_add_13[0][0
 _28 (PruneLowMagnitude)                                         ]']                              
          

2022-12-11 06:02:20.973818: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8200


391/391 [==============================] - 70s 90ms/step - loss: 4.5291 - accuracy: 0.0807 - val_loss: 4.1884 - val_accuracy: 0.1048 - lr: 0.0010
Learning rate:  0.001
Epoch 2/300
391/391 [==============================] - 33s 84ms/step - loss: 3.6930 - accuracy: 0.1837 - val_loss: 3.6925 - val_accuracy: 0.1817 - lr: 0.0010
Learning rate:  0.001
Epoch 3/300
391/391 [==============================] - 33s 84ms/step - loss: 3.3062 - accuracy: 0.2502 - val_loss: 3.3756 - val_accuracy: 0.2402 - lr: 0.0010
Learning rate:  0.001
Epoch 4/300
391/391 [==============================] - 33s 84ms/step - loss: 3.0192 - accuracy: 0.3046 - val_loss: 3.2268 - val_accuracy: 0.2648 - lr: 0.0010
Learning rate:  0.001
Epoch 5/300
391/391 [==============================] - 33s 84ms/step - loss: 2.7956 - accuracy: 0.3495 - val_loss: 3.0947 - val_accuracy: 0.2927 - lr: 0.0010
Learning rate:  0.001
Epoch 6/300
391/391 [==============================] - 33s 84ms/step - loss: 2.6042 - accuracy: 0.3890 - val_los

UsageError: Line magic function `%tensorboard` not found.


## Resnet20

In [ ]:
sparsity_level=[0.6,0.8,0.9]
names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet20_const_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = False, final_sparsity=sparsity_level[x],gpu="T4",\
                    const=True, poly=False,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet20_const_sparsity_sixty
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)


2022-12-11 18:02:07.155762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 18:02:07.303662: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 18:02:07.305477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 18:02:07.308814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d (Pr  (None, 32, 32, 16)  882         ['input_1[0][0]']                
 uneLowMagnitude)                                                                                 
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d[0][0
 alization (PruneLowMagnitude)                                   ]']                              
                                                                        

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Learning rate:  0.001
Epoch 1/300


...

INFO:tensorflow:Assets written to: /tmp/tmp4fssorfs/assets
2022-12-11 21:53:59.566656: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-11 21:53:59.566712: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-11 21:53:59.566883: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp4fssorfs
2022-12-11 21:53:59.623112: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-11 21:53:59.623161: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp4fssorfs
2022-12-11 21:53:59.862144: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-11 21:54:00.500630: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp4fssorfs
2022-12-11 21:54:00.743052: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_PRUNE_resnet20_const_sparsity_ninety_20_T4.h5


INFO:tensorflow:Assets written to: /tmp/tmpaoq4_njx/assets


INFO:tensorflow:Assets written to: /tmp/tmpaoq4_njx/assets
2022-12-11 21:54:49.117906: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-11 21:54:49.117961: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-11 21:54:49.118143: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpaoq4_njx
2022-12-11 21:54:49.173898: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-11 21:54:49.173947: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpaoq4_njx
2022-12-11 21:54:49.415364: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-11 21:54:50.069355: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpaoq4_njx
2022-12-11 21:54:50.319953: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_quant_PRUNE_resnet20_const_sparsity_ninety_20_T4.h5
Size of gzipped pruned Keras model: 246313.00 bytes
Size of gzipped pruned TFlite model: 271858.00 bytes
Size of gzipped pruned and quantized TFlite model: 271881.00 bytes


In [ ]:
sparsity_level=[0.6,0.8,0.9]

names=["sixty","eighty","ninety"]
#for x in sparsity_level:
for x in range(3):
    name="resnet20_poly_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = 0.5, final_sparsity=sparsity_level[x],gpu="T4",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet20_poly_sparsity_sixty
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_63   (None, 32, 32, 16)  882         ['input_4[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_63[0
 alization_57 (PruneLowMagnitud        

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Learning rate:  0.001
Epoch 1/300
  6/391 [..............................] - ETA: 19s - loss: 6.8779 - accuracy: 0.0065 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0288s vs `on_train_batch_end` time: 0.0757s). Check your callbacks.


391/391 [==============================] - ETA: 0s - loss: 4.2706 - accuracy: 0.0875WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 30s 41ms/step - loss: 4.2706 - accuracy: 0.0875 - val_loss: 4.0042 - val_accuracy: 0.1011 - lr: 0.0010
Learning rate:  0.001
Epoch 2/300
391/391 [==============================] - ETA: 0s - loss: 3.5951 - accuracy: 0.1793WARNING:tensorflow:Can save best model only with val_acc available, skipping.


...

INFO:tensorflow:Assets written to: /tmp/tmphwgmuzxa/assets
2022-12-12 04:27:33.042191: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-12 04:27:33.042244: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-12 04:27:33.042411: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmphwgmuzxa
2022-12-12 04:27:33.103679: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-12 04:27:33.103718: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmphwgmuzxa
2022-12-12 04:27:33.467992: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-12 04:27:34.168537: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmphwgmuzxa
2022-12-12 04:27:34.440945: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_PRUNE_resnet20_poly_sparsity_ninety_20_T4.h5


INFO:tensorflow:Assets written to: /tmp/tmpmn9v4_ji/assets


INFO:tensorflow:Assets written to: /tmp/tmpmn9v4_ji/assets
2022-12-12 04:28:30.809125: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-12 04:28:30.809179: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-12 04:28:30.809352: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpmn9v4_ji
2022-12-12 04:28:30.873505: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-12 04:28:30.873560: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpmn9v4_ji
2022-12-12 04:28:31.146418: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-12 04:28:31.839014: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpmn9v4_ji
2022-12-12 04:28:32.126994: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_quant_PRUNE_resnet20_poly_sparsity_ninety_20_T4.h5
Size of gzipped pruned Keras model: 241958.00 bytes
Size of gzipped pruned TFlite model: 269361.00 bytes
Size of gzipped pruned and quantized TFlite model: 269389.00 bytes


In [ ]:
#experimenting block size

In [ ]:
block_sizes=[(2,2),(5,5),(8,8)]

names=["two_two","three_three","five_five"]
#models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_poly_sparsity_block_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = 0.5, final_sparsity=0.8,gpu="V100",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default',block_size=block_sizes[x])
   # models=models+[model1]
    

resnet44_poly_sparsity_block_two_two
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)


2022-12-12 16:51:49.862423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 16:51:50.175916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 16:51:50.176181: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 16:51:50.178462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

ValueError: Block Sparsity can only be used for layers which have 2-dimensional weights.

In [ ]:
freqs=[50,250,500]

names=["freq50","freq250","freq500"]
#models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_poly_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=freqs[x],\
                    initial_sparsity = 0.5, final_sparsity=0.8,gpu="V100",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
   # models=models+[model1]

resnet44_poly_sparsity_freq50
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_45   (None, 32, 32, 16)  882         ['input_2[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_45[0
 alization_43 (PruneLowMagnitud       

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


                                                                                                  
 prune_low_magnitude_activation  (None, 32, 32, 16)  1           ['prune_low_magnitude_add_23[0][0
 _49 (PruneLowMagnitude)                                         ]']                              
                                                                                                  
 prune_low_magnitude_conv2d_52   (None, 32, 32, 16)  4626        ['prune_low_magnitude_activation_
 (PruneLowMagnitude)                                             49[0][0]']                       
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_52[0
 alization_50 (PruneLowMagnitud                                  ][0]']                           
 e)                                                                                               
          

2022-12-12 16:56:06.165324: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8200


391/391 [==============================] - 76s 66ms/step - loss: 4.5204 - accuracy: 0.0849 - val_loss: 4.2407 - val_accuracy: 0.1032 - lr: 0.0010
Learning rate:  0.001
Epoch 2/300
391/391 [==============================] - 24s 61ms/step - loss: 3.7294 - accuracy: 0.1804 - val_loss: 3.7996 - val_accuracy: 0.1844 - lr: 0.0010
Learning rate:  0.001
Epoch 3/300
391/391 [==============================] - 23s 60ms/step - loss: 3.3607 - accuracy: 0.2466 - val_loss: 3.5257 - val_accuracy: 0.2182 - lr: 0.0010
Learning rate:  0.001
Epoch 4/300
391/391 [==============================] - 23s 59ms/step - loss: 3.0711 - accuracy: 0.2994 - val_loss: 3.3310 - val_accuracy: 0.2551 - lr: 0.0010
Learning rate:  0.001
Epoch 5/300
391/391 [==============================] - 24s 60ms/step - loss: 2.8352 - accuracy: 0.3441 - val_loss: 3.0709 - val_accuracy: 0.3005 - lr: 0.0010
Learning rate:  0.001
Epoch 6/300
391/391 [==============================] - 23s 60ms/step - loss: 2.6459 - accuracy: 0.3826 - val_los

...

INFO:tensorflow:Assets written to: /tmp/tmp3h49r44f/assets
2022-12-12 22:45:32.400193: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-12 22:45:32.400273: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-12 22:45:32.400504: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp3h49r44f
2022-12-12 22:45:32.581746: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-12 22:45:32.581824: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp3h49r44f
2022-12-12 22:45:33.374315: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-12 22:45:36.102054: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp3h49r44f
2022-12-12 22:45:36.982072: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_PRUNE_resnet44_poly_sparsity_freq500_44_V100.h5


INFO:tensorflow:Assets written to: /tmp/tmpmyd0ky5y/assets


INFO:tensorflow:Assets written to: /tmp/tmpmyd0ky5y/assets
2022-12-12 22:48:15.868061: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-12 22:48:15.868133: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-12 22:48:15.868345: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpmyd0ky5y
2022-12-12 22:48:16.038045: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-12 22:48:16.038117: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpmyd0ky5y
2022-12-12 22:48:16.817200: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-12 22:48:19.553031: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpmyd0ky5y
2022-12-12 22:48:20.408956: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_quant_PRUNE_resnet44_poly_sparsity_freq500_44_V100.h5
Size of gzipped pruned Keras model: 853812.00 bytes
Size of gzipped pruned TFlite model: 957597.00 bytes
Size of gzipped pruned and quantized TFlite model: 957630.00 bytes


In [ ]:
freqs=[50,250,500]

names=["freq50","freq250","freq500"]
#models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_const_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=freqs[x],\
                    initial_sparsity = False, final_sparsity=0.8,gpu="V100",\
                    const=True, poly=False,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
   # models=models+[model1]

resnet44_const_sparsity_freq50
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_180  (None, 32, 32, 16)  882         ['input_5[0][0]']                
  (PruneLowMagnitude)                                                                             
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_180[
 alization_172 (PruneLowMagnitu      

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 prune_low_magnitude_add_93 (Pr  (None, 16, 16, 32)  1           ['prune_low_magnitude_activation_
 uneLowMagnitude)                                                190[0][0]',                      
                                                                  'prune_low_magnitude_batch_norma
                                                                 lization_192[0][0]']             
                                                                                                  
 prune_low_magnitude_activation  (None, 16, 16, 32)  1           ['prune_low_magnitude_add_93[0][0
 _192 (PruneLowMagnitude)                                        ]']                              
                                                                                                  
 prune_low_magnitude_conv2d_202  (None, 16, 16, 32)  18466       ['prune_low_magnitude_activation_
  (PruneLowMagnitude)                                            192[0][0]']                      
          

391/391 [==============================] - ETA: 0s - loss: 4.7158 - accuracy: 0.0476WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 68s 76ms/step - loss: 4.7158 - accuracy: 0.0476 - val_loss: 4.4072 - val_accuracy: 0.0591 - lr: 0.0010
Learning rate:  0.001
Epoch 2/300
391/391 [==============================] - ETA: 0s - loss: 3.9680 - accuracy: 0.1249WARNING:tensorflow:Can save best model only with val_acc available, skipping.


...

391/391 [==============================] - 27s 70ms/step - loss: 0.3437 - accuracy: 0.9950 - val_loss: 4.8185 - val_accuracy: 0.3775 - lr: 5.0000e-07
Learning rate:  1e-06
Epoch 169/300
391/391 [==============================] - ETA: 0s - loss: 0.3411 - accuracy: 0.9955WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 28s 71ms/step - loss: 0.3411 - accuracy: 0.9955 - val_loss: 4.8218 - val_accuracy: 0.3772 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 170/300
391/391 [==============================] - ETA: 0s - loss: 0.3422 - accuracy: 0.9952WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 27s 69ms/step - loss: 0.3422 - accuracy: 0.9952 - val_loss: 4.8204 - val_accuracy: 0.3778 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 171/300
391/391 [==============================] - ETA: 0s - loss: 0.3416 - accuracy: 0.9956WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 27s 70ms/step - loss: 0.3416 - accuracy: 0.9956 - val_loss: 4.8161 - val_accuracy: 0.3773 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 172/300
391/391 [==============================] - ETA: 0s - loss: 0.3430 - accuracy: 0.9947WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 27s 70ms/step - loss: 0.3430 - accuracy: 0.9947 - val_loss: 4.8182 - val_accuracy: 0.3773 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 173/300
213/391 [===============>..............] - ETA: 11s - loss: 0.3425 - accuracy: 0.9948

In [ ]:
freqs=[50,250,500]

names=["freq50","freq250","freq500"]
#models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet20_poly_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=freqs[x],\
                    initial_sparsity = 0.5, final_sparsity=0.8,gpu="V100",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
   # models=models+[model1]

In [ ]:
sparsity_level=[0.6,0.8,0.9]
names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet20_const_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = False, final_sparsity=sparsity_level[x],gpu="T4",\
                    const=True, poly=False,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n' .join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 12 16:44:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    37W / 300W |      0MiB / 16384MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Training baseline models: 
Resnet 20

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n' .join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec 16 03:24:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W /  70W |      0MiB / 15360MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def resnet_training_baseline(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default',block_size=(1,1)):

    # Default parameters
    batch_size = 128 
    epochs = 300
    data_augmentation = False #True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

#     ####Changes start#####
#     num_images = x_train.shape[0] #* (1 - validation_split)

#     if end_step == 'default':
#         end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

#     #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
#     if poly:
#         pruning_params = {
#               'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
#                                                                        final_sparsity=final_sparsity,
#                                                                        begin_step=begin_step,
#                                                                        end_step=end_step,
#                                                                       frequency=frequency),
#             'block_size': block_size
#         }
#     if const:
#             pruning_params = {
#               'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
#                                                                        begin_step=begin_step,
#                                                                        end_step=end_step,
#                                                                        frequency=frequency),
#                 'block_size': block_size
#         }

#     prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    model = resnet_v1(input_shape=input_shape, depth=depth,num_classes=num_classes)
#    model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

    model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname='/home/kal2257/saved_models/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=';')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback]

    st = time.time()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    #%tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

     
    size=get_gzipped_model_size(save_dir+'/'+model_path) 
    print("Size of gzipped Keras model: %.2f bytes" % (size))
     
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],size], handle)

    return(model,model_path)

In [ ]:
layers=[3,7]
names=["twenty_baseline","forty_baseline"]

for x in range(2):
    name="resnet_"+names[x]
    print(name)
    model1 = resnet_training_baseline(X_train,Y_train,X_test, Y_test,layers=3,frequency=False,\
                    initial_sparsity = False, final_sparsity=False,gpu="T4",\
                    file_name=name,\
                    num_classes=100)
  

resnet_twenty_baseline
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_21 (Conv2D)             (None, 32, 32, 16)   448         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_19 (BatchN  (None, 32, 32, 16)  64          ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                             

In [ ]:
with open('/home/kal2257/resnet_twenty_baseline_20_T4.pickle', 'rb') as handle:
    b1 = pickle.load(handle)
b1

[26.641421794891357,
 [25.468732833862305],
 3.8447303771972656,
 0.14079999923706055,
 3115856]

## Working with mixed precision

In [14]:
gpu_info = !nvidia-smi
gpu_info = '\n' .join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec 16 17:58:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W /  70W |   1338MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

In [16]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [17]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [20]:
inputs = keras.Input(shape=(784,), name='digits')
if tf.config.list_physical_devices('GPU'):
  print('The model will run with 4096 units on a GPU')
  num_units = 4096
else:
  # Use fewer units on CPUs so the model finishes in a reasonable amount of time
  print('The model will run with 64 units on a CPU')
  num_units = 64
dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)


print(dense1.dtype_policy)
print('x.dtype: %s' % x.dtype.name)
# 'kernel' is dense1's variable
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)

outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

The model will run with 64 units on a CPU
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
Outputs dtype: float16


In [21]:
# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Outputs dtype: float32


In [18]:
layers=[3,7]
names=["twenty_prec_test"]

for x in range(1):
    name="resnet_"+names[x]
    print(name)
    model1 = resnet_training_mixed_prec(X_train,Y_train,X_test, Y_test,layers=3,frequency=False,\
                    initial_sparsity = False, final_sparsity=False,gpu="T4",\
                    file_name=name,\
                    num_classes=100)

resnet_twenty_prec_test
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32
<Policy "mixed_float16"

NotFoundError: ignored

In [ ]:
def tflite_conv(model,path,quant=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quant == True:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    pruned_tflite_model = converter.convert()
    with open(path, 'wb') as f:
      f.write(pruned_tflite_model)
    print('Saved pruned TFLite model to:',path)

In [10]:
def resnet_layer_mixed_prec(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    print(conv.dtype_policy)
    print('x.dtype: %s' % x.dtype.name)
    # 'kernel' is dense1's variable
    print('dense1.kernel.dtype: %s' % conv.kernel.dtype.name)
    return x

In [11]:
def resnet_mixed_prec(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer_mixed_prec(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer_mixed_prec(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer_mixed_prec(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer_mixed_prec(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    # x = AveragePooling2D(pool_size=8)(x)
    # y = Flatten()(x)
    # outputs = Dense(num_classes,
    #                 activation='softmax',
    #                 kernel_initializer='he_normal')(y)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal',
                    dtype='float32')(y)
    print('Outputs dtype: %s' % outputs.dtype.name)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
def resnet_training_mixed_prec(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default',block_size=(1,1)):

    # Default parameters
    batch_size = 128 
    epochs = 1
    data_augmentation = False #True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

#     ####Changes start#####
#     num_images = x_train.shape[0] #* (1 - validation_split)

#     if end_step == 'default':
#         end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

#     #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
#     if poly:
#         pruning_params = {
#               'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
#                                                                        final_sparsity=final_sparsity,
#                                                                        begin_step=begin_step,
#                                                                        end_step=end_step,
#                                                                       frequency=frequency),
#             'block_size': block_size
#         }
#     if const:
#             pruning_params = {
#               'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
#                                                                        begin_step=begin_step,
#                                                                        end_step=end_step,
#                                                                        frequency=frequency),
#                 'block_size': block_size
#         }

#     prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    model = resnet_mixed_prec(input_shape=input_shape, depth=depth,num_classes=num_classes)
#    model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

    model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname='/home/kal2257/saved_models/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=';')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback]

    st = time.time()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    #%tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

     
    size=get_gzipped_model_size(save_dir+'/'+model_path) 
    print("Size of gzipped Keras model: %.2f bytes" % (size))
     
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],size], handle)

    return(model,model_path)

# Quantization

In [ ]:
import tempfile
model = tf.keras.models.load_model("/home/kal2257/saved_models/PRUNE_resnet20_poly_sparsity_ninety_20_T4.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

path="/home/kal2257/saved_models/test_quant32.h5"
with open(path, 'wb') as f:
    f.write(tflite_quant_model)
    
get_gzipped_model_size("/home/kal2257/saved_models/test_quant32.h5")

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
PATH = F"/content/gdrive/My Drive/PRUNE_resnet44_const_sparsity_freq50_44_V100.h5"

In [17]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(PATH)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.lite.constants.FLOAT16]
Tflite_quanit_model = converter.convert()

OSError: ignored

In [ ]:
#####scrap########

In [ ]:
model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = False, final_sparsity=0.3,gpu="T4",\
                    const=True, poly=False,file_name='cifar7_test1',\
                    num_classes=100,begin_step=0,end_step='default')

In [ ]:
%load_ext tensorboard
model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=3,frequency=100,\
                    initial_sparsity = False, final_sparsity=0.3,gpu="T4",\
                    const=True, poly=False,file_name='cifar7_test1',\
                    num_classes=100,begin_step=0,end_step='default')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_42   (None, 32, 32, 16)  882         ['input_3[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_42[0
 alization_38 (PruneLowMagnitud                                  ][0

overall training time is 35.96639800071716
each epoch training time is [31.73249888420105]
313/313 [==============================] - 2s 6ms/step - loss: 3.9833 - accuracy: 0.1183
Test loss: 3.9832944869995117
Test accuracy: 0.11829999834299088
tt /home/kal2257/saved_models/cifar7_test1_20_T4
Saved pruned Keras model to: /home/kal2257/saved_models/PRUNE_cifar7_test1_20_T4.h5


INFO:tensorflow:Assets written to: /tmp/tmp2thzh_v7/assets


INFO:tensorflow:Assets written to: /tmp/tmp2thzh_v7/assets
2022-12-10 23:27:25.752954: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-10 23:27:25.753007: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-10 23:27:25.754056: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2thzh_v7
2022-12-10 23:27:25.811024: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-10 23:27:25.811079: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp2thzh_v7
2022-12-10 23:27:26.010131: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-10 23:27:26.079075: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-10 23:27:26.865426: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_PRUNE_cifar7_test1_20_T4.h5


INFO:tensorflow:Assets written to: /tmp/tmpwvclnqeh/assets


INFO:tensorflow:Assets written to: /tmp/tmpwvclnqeh/assets
2022-12-10 23:28:21.566610: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-10 23:28:21.566668: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-10 23:28:21.566852: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpwvclnqeh
2022-12-10 23:28:21.623263: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-10 23:28:21.623318: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpwvclnqeh
2022-12-10 23:28:21.889817: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-10 23:28:22.682462: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpwvclnqeh
2022-12-10 23:28:22.967156: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/kal2257/saved_models/lite_quant_PRUNE_cifar7_test1_20_T4.h5
Size of gzipped pruned Keras model: 839343.00 bytes
Size of gzipped pruned TFlite model: 892934.00 bytes
Size of gzipped pruned and quantized TFlite model: 892955.00 bytes


In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

In [ ]:
with open('/home/kal2257/cifar7_test1_20_T4.pickle', 'rb') as handle:
    b1 = pickle.load(handle)
b1

[35.96639800071716,
 [31.73249888420105],
 3.9832944869995117,
 0.11829999834299088,
 839343,
 892934,
 892955]

In [ ]:
model3 = keras.models.load_model('/home/kal2257/saved_models/PRUNE_cifar5_test1_20_T4.h5')


In [ ]:
model2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_63 (Conv2D)             (None, 32, 32, 16)   448         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_57 (BatchN  (None, 32, 32, 16)  64          ['conv2d_63[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_57 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_57[0]

In [ ]:
model3.get_weights()

[array([[[[ 1.31791592e-01,  5.80994114e-02,  3.31603527e-01,
            7.68987387e-02, -3.04641783e-01, -2.74092122e-03,
           -2.17599317e-01, -1.81094527e-01, -2.20986053e-01,
           -5.89315712e-01, -4.91172522e-01, -2.80829556e-02,
           -1.94892138e-01,  2.27247462e-01,  2.34928757e-01,
            1.73476785e-01],
          [-7.09220171e-02, -3.89235258e-01,  7.98407942e-02,
           -8.37986916e-03,  8.43527764e-02,  9.90333259e-02,
            1.78135782e-02,  3.74973230e-02,  3.60301673e-01,
            2.88770646e-01, -3.37435335e-01, -7.85104409e-02,
            2.62750328e-01,  2.92895108e-01, -5.04738986e-02,
            7.19242617e-02],
          [-2.73054689e-01,  3.28764826e-01,  4.08030450e-01,
           -2.07959801e-01,  3.77648622e-01,  2.56940186e-01,
           -4.75658506e-01, -1.39780447e-01, -9.34910998e-02,
           -1.73302621e-01, -1.96264178e-01, -3.23021486e-02,
            3.66190404e-01,  9.89206582e-02,  2.14506701e-01,
            

In [ ]:
%load_ext tensorboard

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

In [ ]:
pickle.dump([training_time,epoch_times,scores[0],scores[1],pruned,tflite_pruned,tflite_quant_pruned], handle)
